# Libs

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

from mealpy import FloatVar, IntegerVar, ES, PSO

# Load Dataset

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 784).astype("float32") / 255.0
x_test = x_test.reshape(-1, 784).astype("float32") / 255.0

print("Training data shape:", x_train.shape)
print("Test data shape:", x_test.shape)

# Set functions

In [ ]:
def build_autoencoder_classifier(params):
    hidden_layers = int(round(params[0]))
    neurons = int(round(params[1]))
    lr = float(params[2])

    input_layer = Input(shape=(784,))
    x = input_layer
    for _ in range(hidden_layers):
        x = Dense(neurons, activation='relu')(x)
    latent = Dense(32, activation='relu')(x)

    # Decoder
    x = latent
    for _ in range(hidden_layers):
        x = Dense(neurons, activation='relu')(x)
    output_layer = Dense(784, activation='sigmoid')(x)

    autoencoder = Model(input_layer, output_layer)
    encoder = Model(input_layer, latent)
    autoencoder.compile(optimizer=Adam(lr), loss='mse')
    return autoencoder, encoder

def build_classifier(encoder):
    for layer in encoder.layers:
        layer.trainable = False
    input_latent = encoder.input
    x = encoder.output
    x = Dense(64, activation='relu')(x)
    output = Dense(10, activation='softmax')(x)
    model = Model(input_latent, output)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def objective_function(solution):
    try:
        # Sanitização dos parâmetros para garantir que estão dentro dos limites e possuem valores válidos
        hidden_layers = int(round(np.clip(solution[0], 1, 3)))
        neurons = int(round(np.clip(solution[1], 16, 256)))
        lr = float(np.clip(solution[2], 0.0001, 0.01))

        # Debug
        print(f"Avaliando: Camadas={hidden_layers}, Neurônios={neurons}, LR={lr:.5f}")

        # Construção do modelo
        autoencoder, encoder = build_autoencoder_classifier([hidden_layers, neurons, lr])
        autoencoder.fit(x_train, x_train, epochs=5, batch_size=256, verbose=0)

        classifier = build_classifier(encoder)
        classifier.fit(x_train, y_train, epochs=5, batch_size=256, verbose=0)
        preds = classifier.predict(x_test)
        acc = accuracy_score(y_test, np.argmax(preds, axis=1))
        return 1 - acc  # menor é melhor
    except Exception as e:
        print("Erro ao avaliar indivíduo:", solution)
        print("Detalhes:", str(e))
        return 1.0  # penalização forte

# Set params for the problem

In [ ]:
problem_dict = {
    "bounds": [
        IntegerVar(lb=1, ub=5, name="hidden_layers"),
        IntegerVar(lb=16, ub=256, name="neurons"),
        FloatVar(lb=0.0001, ub=0.01, name="lr"),
    ],
    "minmax": "min",
    "obj_func": objective_function
}

# ES

## Run ES

In [ ]:
model = ES.OriginalES(epoch=15, pop_size=20, lamda=0.75, verbose=True)
g_best = model.solve(problem_dict)

## ES Results

In [ ]:
print("\nMelhor solução encontrada:")
print(f"Camadas: {int(round(g_best.solution[0]))}")
print(f"Neurônios por camada: {int(round(g_best.solution[1]))}")
print(f"Taxa de aprendizado: {g_best.solution[2]:.5f}")
print(f"Fitness (1 - acurácia): {g_best.target.fitness:.5f}")

plt.plot(model.history.list_global_best_fit)
plt.title("Convergência do Evolution Strategy (MEALpy)")
plt.xlabel("Geração")
plt.ylabel("Fitness (1 - acurácia)")
plt.grid(True)
plt.show()

## ES Metrics

In [ ]:
best_layers = int(round(g_best.solution[0]))
best_neurons = int(round(g_best.solution[1]))
best_lr = float(g_best.solution[2])

print("\n--- Avaliação final com o melhor modelo ---")
print(f"Camadas ocultas: {best_layers}")
print(f"Neurônios por camada: {best_neurons}")
print(f"Learning rate: {best_lr:.5f}")

# Reconstruir e treinar
autoencoder, encoder = build_autoencoder_classifier([best_layers, best_neurons, best_lr])
autoencoder.fit(x_train, x_train, epochs=5, batch_size=256, verbose=0)

classifier = build_classifier(encoder)
classifier.fit(x_train, y_train, epochs=5, batch_size=256, verbose=0)

# Previsões e métricas
y_pred = classifier.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

acc = accuracy_score(y_test, y_pred_classes)
prec = precision_score(y_test, y_pred_classes, average='macro')
rec = recall_score(y_test, y_pred_classes, average='macro')
f1 = f1_score(y_test, y_pred_classes, average='macro')

print(f"\nAcurácia final:  {acc:.4f}")
print(f"Precisão:        {prec:.4f}")
print(f"Revocação:       {rec:.4f}")
print(f"F1-Score:        {f1:.4f}")

# PSO

## Run PSO

In [ ]:
model = PSO.OriginalPSO(epoch=15, pop_size=20, c1=2.05, c2=2.5, w=0.4, verbose=True)
g_best = model.solve(problem_dict)

## PSO Results

In [ ]:
print("\nMelhor solução encontrada:")
print(f"Camadas: {int(round(g_best.solution[0]))}")
print(f"Neurônios por camada: {int(round(g_best.solution[1]))}")
print(f"Taxa de aprendizado: {g_best.solution[2]:.5f}")
print(f"Fitness (1 - acurácia): {g_best.target.fitness:.5f}")

plt.plot(model.history.list_global_best_fit)
plt.title("Convergência do PSO (MEALpy - OriginalPSO)")
plt.xlabel("Geração")
plt.ylabel("Fitness (1 - Acurácia)")
plt.grid(True)
plt.show()

## PSO Metrics

In [ ]:
best_layers = int(round(g_best.solution[0]))
best_neurons = int(round(g_best.solution[1]))
best_lr = float(g_best.solution[2])

print("\n--- Avaliação final com o melhor modelo ---")
print(f"Camadas ocultas: {best_layers}")
print(f"Neurônios por camada: {best_neurons}")
print(f"Learning rate: {best_lr:.5f}")

# Reconstruir e treinar
autoencoder, encoder = build_autoencoder_classifier([best_layers, best_neurons, best_lr])
autoencoder.fit(x_train, x_train, epochs=5, batch_size=256, verbose=0)

classifier = build_classifier(encoder)
classifier.fit(x_train, y_train, epochs=5, batch_size=256, verbose=0)

# Previsões e métricas
y_pred = classifier.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

acc = accuracy_score(y_test, y_pred_classes)
prec = precision_score(y_test, y_pred_classes, average='macro')
rec = recall_score(y_test, y_pred_classes, average='macro')
f1 = f1_score(y_test, y_pred_classes, average='macro')

print(f"\nAcurácia final:  {acc:.4f}")
print(f"Precisão:        {prec:.4f}")
print(f"Revocação:       {rec:.4f}")
print(f"F1-Score:        {f1:.4f}")